In [56]:
import numpy as np
import matplotlib.pyplot as plt
from hamming_distance_distribution import hamming_distance
from annealing import is_connected, adjacency_from_couplings, degree_of_nodes

In [57]:
def annealing_by_degree(fname):
    pass

In [58]:
# ---------- Parameters for reading data file -------------
path = "./annealing_data/"

N = 5               # number of users
M = 4               # length of id-sequence for every user
K = 100              # number of antennas
xi = 0              # std of thermal noise

In [59]:
# ----------------- Program -----------------
# check the degree distribution of different neglection thresolds
fname = path + f"neglection_thres_{0.15}_N_5_M_4_K_100_xi_0.npy"
data = np.load(fname, allow_pickle=True).item()

i = 15
print(data["J_n"][i])
print(data["where_n"][i])
print(degree_of_nodes(adjacency_from_couplings(data["J_n"][i])))
print(is_connected(adjacency_from_couplings(data["J_n"][i])))

[[ 0.          0.          0.          0.         -0.46181011]
 [ 0.          0.          0.         -0.21812017  0.        ]
 [ 0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.        ]]
(array([0, 0, 0, 1, 1, 2, 2, 3]), array([1, 2, 3, 2, 4, 3, 4, 4]))
[1 1 0 1 1]
False
